# Dantzig-Wolfe Decomposition

In large-scale optimization complicating constraints are constraints that couple different units together. THe rest of the constraints are easy constraints, that refer to each unit. When we remove the complicating constraint, the rest of the constraints become simple.

## Steps for Decomposition

1. Partition the constraints into complicating constraints and easy constraints.
2. Use extreme point representation for P to represent the easy constraints.
3. Substitute P into the original problem in lambda space.
4. Apply column generation to solve master problem.

## Explanation

Consider the following general linear program:

$$
\min{c^Tx} \\
s.t. \\
Dx = b_0 \\
Fx = b \\
x \geq 0
$$

Reformulate it as:

$$
\min{c^Tx} \\
s.t. \\
Dx = b_0 \\
x \in P
$$

Where $P = \{x : Fx = b, x \geq 0\}$, a polytope. P can more specifically be rewritten as the following constraints:

$$
x = \sum^N_{i=1}\lambda x^i \\
\sum^N_{i=1} \lambda_i = 1 \\
\lambda_i \geq 0, \forall i \in \{1...N\}
$$

We substitute P back into the original LP:

$$
(MP) \min{\sum^N_{i=1}\lambda_i(c^Tx^i)} \\
s.t. \\
\sum^N_{i=1}\lambda_i(Dx^i) = b_0 \\
\sum^N_{i=1}\lambda_i = 1 \\
\lambda_i \geq 0, \forall i \in \{1...N\}
$$

We can expand this problem out in matrix form which looks like this:

$$
\min \space [c^Tx^1 \space c^Tx^2 \space \cdots \space c^Tx^N] 
\begin{bmatrix}
           \lambda_{1} \\
           \lambda_{2} \\
           \vdots \\
           \lambda_{N}
\end{bmatrix} \\
s.t. \\
\begin{bmatrix}
    Dx^1 & Dx^2 & \cdots & Dx^N \\
    1 & 1 & \cdots & 1
\end{bmatrix}
\begin{bmatrix}
           \lambda_{1} \\
           \lambda_{2} \\
           \vdots \\
           \lambda_{N}
\end{bmatrix} = 
\begin{bmatrix}
           b_0 \\
           1 \\
\end{bmatrix} \\
\lambda_i \geq 0, \forall i = 1,..., N
$$

Let's note a few things:
* The $x_i$'s are extreme points of the polytope P.
* There are only a few constraints
* The number of extreme points blow up depending on dimensionality of P ($2^N$).

We apply column generation by checking the optimality of $\lambda$ using the dual solutions $\hat{y}$ and $\hat{r}$. Each polyvector in the RMP represents a another optimization problem: reduced pricing problem. We formulate it as follows:

$$
\hat{Z} = \min{(c^Tx^i) - \hat{y}^T(Dx^i)-\hat{r}}
$$

We construct the following logic:

* If the minimum reduced cost, Z is less than 0, we add coumn to I.
* Else we stop and come to an optimal solution.

$Z$ becomes an easily problem to solve because we can apply the simplex method to the following LP formulation of Z:

$$
\min{(c^T-\hat{y}^TD)x)-\hat{r}} \\
s.t. \\
Fx = b \\
x \geq 0
$$

At a high level, the steps to applying column generation are:
1. Start with a susbet of columns I to form the restricted master problem (RMP)
2. Solve RMP and get the optimal dual variables, $\hat{y}$ and $\hat{r}$
3. Check optimality of the RMPs primal solution.

Dantzig-Wolfe is a coordinated distributed optimization problem. The RMP handles complicated coupling constraints by finding dual variables. Then it passes the dual variables to each unit (pricing problem). Based off the results of the pricing problem, it informs the RMP whether it is optimal or not. This process repeats.